In [1]:
from QuantitativeCleaner import *

In [2]:
# Read Path
optimized_path = "F:/Data/Processing_data/OptimizedTrackData.parquet"

# Write paths
group_dir = "F:/Data/Processing_data/injury_output"
main_dir = "F:/Data/Processing_data/"

In [3]:
# optimize_injury_data()

In [4]:
# process_file(optimized_path, group_dir)

In [6]:
#### Tracking Output
def tracking_injuries(group_dir, main_dir):
    from DataHandler import data_loader
    import polars as pl # type: ignore
    import os

    # Read in the PlayKeys from the injury file to isolate PlayKeys associated with injury paths
    injuryPlayKeys = data_loader('injuries')
    PlayKeys = injuryPlayKeys.select("PlayKey").unique().with_columns([
            pl.col("PlayKey").cast(pl.Utf8)
            ])

    # Initialize an empty list to store dataframes
    filtered_dfs = []

    # Iterate through files in the directory
    for file in os.listdir(group_dir):
        if file.startswith("group_"):
            file_path = os.path.join(group_dir, file)
            
            # Read the Parquet file
            df = pl.read_parquet(file_path)
            
            # Inner join with unique_gsisid to filter rows - Using inner since this is going to Viz and we need all data there. 
            filtered_df = df.join(PlayKeys, on="PlayKey", how="inner")
            
            # Append to the list of filtered dataframes
            filtered_dfs.append(filtered_df)

    # Concatenate all filtered dataframes
    final_df = pl.concat(filtered_dfs)

    # Save the concatenated dataframe
    final_df.write_parquet(os.path.join(main_dir, "TrackingInjuries.parquet"))

    print("Processing complete. Filtered summary dataframe saved as 'TrackingInjuries.parquet'")

In [7]:
tracking_injuries(group_dir, main_dir)

StringCacheMismatchError: cannot compare categoricals coming from different sources, consider setting a global StringCache.

Help: if you're using Python, this may look something like:

    with pl.StringCache():
        # Initialize Categoricals.
        df1 = pl.DataFrame({'a': ['1', '2']}, schema={'a': pl.Categorical})
        df2 = pl.DataFrame({'a': ['1', '3']}, schema={'a': pl.Categorical})
    # Your operations go here.
    pl.concat([df1, df2])

Alternatively, if the performance cost is acceptable, you could just set:

    import polars as pl
    pl.enable_string_cache()

on startup.